###  Name: Chinedu Onyeka

The airshow.mp4 video has been downloaded and resides in the attcoding folder

In [1]:
abs_path = 'C:\\Users\\exper\\OneDrive\\Documents\\Jobs\\Interviews\\ATT\\attcoding'
file_name = "airshow.mp4"

In [2]:
class CutVideo:
    
    def __init__(self, file_name, abs_path):
        """
        file_name: filename(with mp4 extention) of the video."
        abs_path: absolute path where the file_name is located
        """
        self.file_name = file_name
        self.abs_path = abs_path
        
    
    def create_folder(self, folder_name):
        """This function creates a folder called 'folder_name' in the abs_path given
        
        parameter:
        =========
            folder_name: name of the folder you want to create
        """
        import os
        path_name = self.abs_path + "\\" + folder_name
        if not os.path.exists(path_name):
            os.makedirs(path_name)
    
    def clip_duration(self):
        """
        This function returns the duration of a given mp4 video

        parameter:
        ==========
            file_name: filename("with mp4 extention") of the video that you want to get the duration for.
        output:
        =======
            duration: duration in seconds for the video file name entered.
        """
        import os
        from moviepy.editor import VideoFileClip
        clip_path = self.abs_path + "\\" + self.file_name
        clip = VideoFileClip(clip_path)
        clip_duration = clip.duration
        return clip_duration
    
    def cut_clip(self, video_clips_folder):
        import os
        from moviepy.video.io.ffmpeg_tools import ffmpeg_extract_subclip
        """
        This function cuts a video returns about 1 minute clips of a given mp4 video

        parameter:
        ==========
            video_clips_folder: folder name where you want the video clips to be stored. Should exist in current working dir
        output:
        =======
            1 minute clips: one minute clips of the video name given
        """
        video_name = self.abs_path + "\\" + self.file_name
        
        path = self.abs_path + "\\" + video_clips_folder + "\\" # where the video clips will be stored

        # cut the videos into about 1 minute clips
        start_time = 0
        while start_time < self.clip_duration():
            if self.clip_duration() - start_time > 60:
                end_time = start_time + 60
            else:
                end_time = self.clip_duration()
            target_name = path + str(round(start_time,0)) + "thframe.mp4"

            ffmpeg_extract_subclip(video_name, start_time, end_time, targetname= target_name)

            start_time = end_time
            
    def clips_list(self, video_clips_folder):
        """
        This function returns a list of all videos in the given folder

        parameter:
        ==========
            video_clips_folder: folder name where you want the list of video clips. Should exist in current working dir
        output:
        =======
            list: list of all video clips in the folder
        """
        import os
        clips_list = []
        # Iterate directory
        dir_path = self.abs_path + "\\" + video_clips_folder + "\\"
        for path in os.listdir(dir_path):
            # check if current path is a file
            if os.path.isfile(os.path.join(dir_path, path)):
                clips_list.append(path)
        return clips_list
    
    def clips_location_list(self, video_clips_folder):
        """
        This function returns a list of absolute path of all videos in the given folder

        parameter:
        ==========
            video_clips_folder: folder name where you want the list of video clips location.
        output:
        =======
            list: list of locations of all video clips in the folder
        """
        import os
        clips_location_list = []
        # Iterate directory
        dir_path = video_clips_folder + "\\"
        for path in os.listdir(dir_path):
            # check if current path is a file
            if os.path.isfile(os.path.join(dir_path, path)):
                clips_location_list.append(dir_path + path)
        return clips_location_list

In [3]:
airshow = CutVideo(file_name, abs_path)

Get the total duration of the video you want to cut into clips

In [4]:
airshow_duration = airshow.clip_duration()
print(airshow_duration)

309.88


Cut the video into about 1 minute clips

In [5]:
# Create New folder "video_clips"
folder_name = "video_clips"
airshow.create_folder(folder_name)

# Cut the video "airshow.mp4" into 1 minute clips and save inside the "video_clips" folder

airshow.cut_clip(folder_name)

Moviepy - Running:
>>> "+ " ".join(cmd)
Moviepy - Command successful
Moviepy - Running:
>>> "+ " ".join(cmd)
Moviepy - Command successful
Moviepy - Running:
>>> "+ " ".join(cmd)
Moviepy - Command successful
Moviepy - Running:
>>> "+ " ".join(cmd)
Moviepy - Command successful
Moviepy - Running:
>>> "+ " ".join(cmd)
Moviepy - Command successful
Moviepy - Running:
>>> "+ " ".join(cmd)
Moviepy - Command successful


Prepare the data required to populate the postgres table

In [6]:
# prepare the data required to update the postgres db
clip_list = airshow.clips_list(folder_name)
clip_loc_list = airshow.clips_location_list(folder_name)

def clip_duration(video_name):
    """
    This function returns the duration of a given mp4 video
    
    parameter:
    ==========
        video_name: filename("with mp4 extention") of the video that you want to get the duration for.
    output:
    =======
        duration: duration in seconds for the video file name entered.
    """
    from moviepy.editor import VideoFileClip
    clip = VideoFileClip(video_name)
    clip_duration = clip.duration
    return clip_duration

clip_dur_list = []
for vid in clip_loc_list:
    duration = clip_duration(vid)
    clip_dur_list.append(duration)



print("clip_list_name: ", clip_list)
print()
print("clip_duration_list: ", clip_dur_list)
print()
print("clip_location_list: ", clip_loc_list)

clip_list_name:  ['0thframe.mp4', '120thframe.mp4', '180thframe.mp4', '240thframe.mp4', '300thframe.mp4', '60thframe.mp4']

clip_duration_list:  [60.03, 60.61, 60.53, 60.49, 10.29, 60.96]

clip_location_list:  ['video_clips\\0thframe.mp4', 'video_clips\\120thframe.mp4', 'video_clips\\180thframe.mp4', 'video_clips\\240thframe.mp4', 'video_clips\\300thframe.mp4', 'video_clips\\60thframe.mp4']


In [7]:
# create table query
create_table_query = '''
        CREATE TABLE IF NOT EXISTS video_data (
            clip_name VARCHAR(25),
            clip_file_extension VARCHAR(5),
            clip_duration NUMERIC (7, 2),
            clip_location VARCHAR(200),
            insert_timestamp TIMESTAMP
        );
        '''

Make connection to postgress database

In [8]:
# create a connection to postgress
import psycopg2 as pg2
from getpass import getpass
password = getpass(prompt = "Enter your database password: ")
conn = pg2.connect(database='learning', user='postgres',password = password)

Enter your database password: ········


In [9]:
# Start the cursor to be ready to query
cur = conn.cursor()

Create video_data table in postgres 

In [10]:
# Create video_data table in postgres

cur.execute("DROP TABLE IF EXISTS video_data")
cur.execute(create_table_query)
conn.commit()

Insert the records into postgres

In [11]:
# insert the values into the video_data table
from datetime import datetime, timezone

insert = """INSERT INTO video_data VALUES (%s, %s, %s, %s, %s) """

values0 = (f"{clip_list[0][:-4]}", f"{clip_list[0][-3:]}", clip_dur_list[0], f"{clip_loc_list[0]}", datetime.now(timezone.utc))
values1 = (f"{clip_list[1][:-4]}", f"{clip_list[1][-3:]}", clip_dur_list[1], f"{clip_loc_list[1]}", datetime.now(timezone.utc))
values2 = (f"{clip_list[2][:-4]}", f"{clip_list[2][-3:]}", clip_dur_list[2], f"{clip_loc_list[2]}", datetime.now(timezone.utc))
values3 = (f"{clip_list[3][:-4]}", f"{clip_list[3][-3:]}", clip_dur_list[3], f"{clip_loc_list[3]}", datetime.now(timezone.utc))
values4 = (f"{clip_list[4][:-4]}", f"{clip_list[4][-3:]}", clip_dur_list[4], f"{clip_loc_list[4]}", datetime.now(timezone.utc))
values5 = (f"{clip_list[5][:-4]}", f"{clip_list[5][-3:]}", clip_dur_list[5], f"{clip_loc_list[5]}", datetime.now(timezone.utc))

cur.execute(insert, values0)
cur.execute(insert, values1)
cur.execute(insert, values2)
cur.execute(insert, values3)
cur.execute(insert, values4)
cur.execute(insert, values5)
conn.commit()

Read the records in postgres database into a pandas dataframe

In [12]:
# read table in postgres into pandas dataframe
import pandas.io.sql as psql
video_data_table = psql.read_sql_query("SELECT * FROM video_data", conn)

# close the connection
conn.close()

In [13]:
video_data_table

,clip_name,clip_file_extension,clip_duration,clip_location,insert_timestamp
0,0thframe,mp4,60.03,video_clips\0thframe.mp4,2022-08-24 15:07:19.613144
1,120thframe,mp4,60.61,video_clips\120thframe.mp4,2022-08-24 15:07:19.613144
2,180thframe,mp4,60.53,video_clips\180thframe.mp4,2022-08-24 15:07:19.614141
3,240thframe,mp4,60.49,video_clips\240thframe.mp4,2022-08-24 15:07:19.614141
4,300thframe,mp4,10.29,video_clips\300thframe.mp4,2022-08-24 15:07:19.614141
5,60thframe,mp4,60.96,video_clips\60thframe.mp4,2022-08-24 15:07:19.614141


Save the pandas dataframe into a csv file in folder report

In [14]:
# create folder "report"

def create_folder(folder_name):
    """This function creates a folder called 'folder_name' in the current working directory

    parameter:
    =========
        folder_name: name of the folder you want to create
    """
    import os
    path_name = folder_name
    if not os.path.exists(path_name):
        os.makedirs(path_name)
        
create_folder("report")

In [15]:
# save the pandas dataframe into a csv file located in "report" folder
path_name = "report/generated_video_files.csv"
video_data_table.to_csv(path_name)